<a href="https://colab.research.google.com/github/Akshatha7710/telco-customer-churn/blob/main/telco_customer_churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ---------------------------
# Full Upgraded Coursework Script (Optimized for High Grade)
# ---------------------------

# Core imports
import os, json
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score, confusion_matrix

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.neural_network import MLPClassifier

# Optional libraries
try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
except Exception:
    XGBOOST_AVAILABLE = False

try:
    import shap
    SHAP_AVAILABLE = True
except Exception:
    SHAP_AVAILABLE = False

try:
    from imblearn.over_sampling import SMOTE
    IMBLEARN_AVAILABLE = True
except Exception:
    IMBLEARN_AVAILABLE = False

# Random state
RANDOM_STATE = 42

# Create outputs folder
os.makedirs("outputs", exist_ok=True)

# ---------------------------
# Load Dataset
# ---------------------------
data_path = "data/Telco-Customer-Churn.csv"
if not os.path.exists(data_path):
    print(f"[WARN] Dataset not found at {data_path}, creating synthetic data")
    from sklearn.datasets import make_classification
    X_synthetic, y_synthetic = make_classification(
        n_samples=2000, n_features=20, n_informative=10,
        n_redundant=5, n_classes=2, weights=[0.69, 0.31],
        random_state=RANDOM_STATE
    )
    df = pd.DataFrame(X_synthetic, columns=[f"feature_{i}" for i in range(20)])
    df["Churn"] = np.where(y_synthetic==1, "Yes", "No")
else:
    df = pd.read_csv(data_path)

print(f"[INFO] Dataset loaded. Shape: {df.shape}")

# ---------------------------
# Ensure unique CustomerID
# ---------------------------
if 'customerID' not in df.columns:
    df.insert(0, 'customerID', ['CUST' + str(i).zfill(5) for i in range(1, len(df)+1)])
else:
    # Ensure no duplicates
    if df['customerID'].duplicated().any():
        df['customerID'] = ['CUST' + str(i).zfill(5) for i in range(1, len(df)+1)]


# ---------------------------
# EDA (Task 1)
# ---------------------------
print("----- EDA -----")

if 'TotalCharges' in df.columns:
    df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# Show a sample that includes customerID for the report appendix, then drop ID before model prep
print(df.head())   # this prints the sample with customerID present (if it exists)
if 'customerID' in df.columns:
    # drop before preprocessing (customerID is an identifier, not a feature)
    df.drop(columns=['customerID'], inplace=True)


# 1. Basic checks
print(df.head())
print("\nMissing values:\n", df.isnull().sum())
print("\nTarget distribution:\n", df["Churn"].value_counts())

# 2. Target distribution
plt.figure(figsize=(6,4))
sns.countplot(x='Churn', data=df)
plt.title('Churn Distribution')
plt.savefig("outputs/churn_distribution.png")
plt.close()

# 3. Numeric features EDA
numerical_cols = df.select_dtypes(include=['int64','float64']).columns.tolist()
for col in numerical_cols:
    plt.figure(figsize=(8,5))
    sns.histplot(df[col], bins=30, kde=True)
    plt.title(f'{col} Histogram')
    plt.savefig(f"outputs/{col}_hist.png")
    plt.close()

    # Boxplot for outlier detection
    plt.figure(figsize=(8,5))
    sns.boxplot(x='Churn', y=col, data=df)
    plt.title(f'{col} Boxplot by Churn')
    plt.savefig(f"outputs/{col}_boxplot.png")
    plt.close()

# 4. Correlation heatmap
plt.figure(figsize=(12,10))
sns.heatmap(df[numerical_cols].corr(), annot=True, fmt=".2f", cmap='coolwarm')
plt.title("Correlation Heatmap of Numeric Features")
plt.savefig("outputs/correlation_heatmap.png")
plt.close()

print("[INFO] EDA plots saved in outputs/")

# ---------------------------
# Preprocessing (Task 2)
# ---------------------------
def preprocess(df):
    categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()
    if "Churn" in categorical_cols:
        categorical_cols.remove("Churn")

    numerical_cols = df.select_dtypes(include=["int64", "float64"]).columns.tolist()

    cat_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    preprocessor = ColumnTransformer([
        ('num', num_pipeline, numerical_cols),
        ('cat', cat_pipeline, categorical_cols)
    ], remainder='passthrough')

    X = df.drop(columns="Churn")
    y = df["Churn"].map({"No":0, "Yes":1})

    X_trans = preprocessor.fit_transform(X)

    # Handle imbalance
    if IMBLEARN_AVAILABLE:
        sm = SMOTE(random_state=RANDOM_STATE)
        X_trans, y = sm.fit_resample(X_trans, y)

    return X_trans, y, preprocessor, categorical_cols, numerical_cols

X, y, preprocessor, cat_cols, num_cols = preprocess(df)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=RANDOM_STATE
)

# ---------------------------
# Models & Hyperparameter Tuning
# ---------------------------
models = {
    "dummy": DummyClassifier(strategy="most_frequent"),
    "random_forest": RandomForestClassifier(random_state=RANDOM_STATE, n_estimators=100),
}

if XGBOOST_AVAILABLE:
    models["xgboost"] = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=RANDOM_STATE)

# Tuned Decision Tree
dt_model = DecisionTreeClassifier(random_state=RANDOM_STATE)
dt_param_grid = {'max_depth':[5,10,15], 'min_samples_leaf':[5,10]}
tuned_dt = GridSearchCV(dt_model, dt_param_grid, cv=3, scoring='f1', n_jobs=1)
models["tuned_decision_tree"] = tuned_dt

# Tuned Neural Network
nn_model = MLPClassifier(max_iter=300, random_state=RANDOM_STATE, early_stopping=True)
nn_param_grid = {'hidden_layer_sizes':[(32,16),(64,32),(64,)], 'alpha':[0.0001,0.001]}
tuned_nn = GridSearchCV(nn_model, nn_param_grid, cv=3, scoring='f1', n_jobs=1)
models["tuned_neural_network"] = tuned_nn

# ---------------------------
# Training & Evaluation
# ---------------------------
results = {}
for name, model in models.items():
    print(f"[INFO] Training {name}")
    model.fit(X_train, y_train)
    best_model = model.best_estimator_ if hasattr(model,'best_estimator_') else model

    y_pred = best_model.predict(X_test)
    y_proba = best_model.predict_proba(X_test)[:,1] if hasattr(best_model,"predict_proba") else y_pred

    results[name] = {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred, zero_division=0),
        "recall": recall_score(y_test, y_pred, zero_division=0),
        "f1": f1_score(y_test, y_pred, zero_division=0),
        "roc_auc": roc_auc_score(y_test, y_proba),
        "pr_auc": average_precision_score(y_test, y_proba),
        "confusion_matrix": confusion_matrix(y_test, y_pred).tolist()
    }

    if hasattr(model,'best_params_'):
        results[name]["best_params"] = model.best_params_
        print(f"       Best Parameters: {model.best_params_}")

    # Confusion matrix plot
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(f"{name} Confusion Matrix")
    plt.savefig(f"outputs/{name}_confusion_matrix.png")
    plt.close()

# Save results
with open("outputs/results.json","w") as f:
    json.dump(results,f,indent=4)

print("[INFO] Training complete. Results saved to outputs/results.json")

# ---------------------------
# Feature Importance & Model Plots
# ---------------------------
try:
    feature_names = preprocessor.get_feature_names_out()
except AttributeError:
    feature_names = [f"feature_{i}" for i in range(X.shape[1])]

# Feature importance for tree models
for name in ["tuned_decision_tree","random_forest","xgboost"]:
    if name in models:
        model_to_check = models[name].best_estimator_ if hasattr(models[name],'best_estimator_') else models[name]
        if hasattr(model_to_check,"feature_importances_"):
            fi = model_to_check.feature_importances_
            sorted_idx = fi.argsort()[-20:]
            plt.figure(figsize=(12,8))
            plt.barh(feature_names[sorted_idx], fi[sorted_idx])
            plt.title(f"{name} Feature Importances")
            plt.tight_layout()
            plt.savefig(f"outputs/{name}_feature_importance.png")
            plt.close()

# MLP loss curve
if "tuned_neural_network" in models:
    nn_model = models["tuned_neural_network"].best_estimator_
    if hasattr(nn_model,"loss_curve_"):
        plt.figure(figsize=(8,5))
        plt.plot(nn_model.loss_curve_)
        plt.title("Tuned Neural Network Training Loss Curve")
        plt.xlabel("Iteration")
        plt.ylabel("Loss")
        plt.grid(True)
        plt.tight_layout()
        plt.savefig("outputs/nn_loss_curve.png")
        plt.close()

# ---------------------------
# SHAP Explanations (Ethics Discussion)
# ---------------------------
if SHAP_AVAILABLE and XGBOOST_AVAILABLE:
    print("[INFO] Generating SHAP explanations")
    xgb_model = models["xgboost"].best_estimator_ if hasattr(models["xgboost"],'best_estimator_') else models["xgboost"]
    if hasattr(xgb_model,'feature_importances_'):
        explainer = shap.TreeExplainer(xgb_model)
        X_sample = X_test[:200]
        feature_names_list = feature_names.tolist()
        try:
            shap_values = explainer.shap_values(X_sample)
            X_sample_df = pd.DataFrame(X_sample, columns=feature_names_list)
            shap.summary_plot(shap_values, X_sample_df, show=False)
            plt.savefig("outputs/shap_summary.png")
            plt.close()
        except Exception as e:
            print(f"[WARN] SHAP plotting failed: {e}")

print("[INFO] All outputs saved in: outputs/")
print("[INFO] AI Ethics: SMOTE used to address class imbalance, SHAP for explainability.")
print("[INFO] Post-deployment: Monitor model drift, retrain periodically, log misclassifications for fairness analysis.")



[WARN] Dataset not found at data/Telco-Customer-Churn.csv, creating synthetic data
[INFO] Dataset loaded. Shape: (2000, 21)
----- EDA -----
  customerID  feature_0  feature_1  feature_2  feature_3  feature_4  \
0  CUST00001   0.836084  -0.406200   0.985852  -1.098696   0.896335   
1  CUST00002  -0.540182  -5.246919  -0.296145  -1.540667  -0.455911   
2  CUST00003   0.958855  -0.294964  -0.599162  -2.020151   0.696867   
3  CUST00004   2.880514   0.903751   0.953711  -1.972112  -1.039654   
4  CUST00005  -1.945330  -4.153046  -0.808245   0.836231  -1.757355   

   feature_5  feature_6  feature_7  feature_8  ...  feature_11  feature_12  \
0   0.755028  -1.412918  -0.556318   0.570253  ...   -0.897290   -0.326756   
1  -0.262313  -0.244954   2.069706  -2.330663  ...    2.734267    3.698911   
2   2.054202   1.264476  -0.387519  -0.821913  ...    0.722353    1.012499   
3  -0.190190   0.385689  -1.639310   0.116479  ...   -2.197232    2.682806   
4  -0.974836  -1.006793  -2.172324   0.4767